# <font color=steelblue>DS-03 Data Science und künstliche Intelligenz</font>

Deployment eines Jupyter Notebooks als WebApp mit Hilfe von Anvil

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import PIL
from sklearn.neural_network import MLPClassifier


Unser Modell laden:

In [2]:
import pickle

filename = "ZiffernModellDeployment.pkl"

with open(filename, "rb") as file:
    savedModel = pickle.load(file)


Codeblock für Debug-Zwecke:

In [3]:
#für Testzwecke
import PIL
from PIL import Image
img = Image.open("Bilder/3.png")

img = PIL.ImageOps.grayscale(img)

img.thumbnail((8,8), Image.Resampling.BICUBIC)

arr = np.array(img)

arr = arr/255
arr = 1- arr
arr = arr*16

print(arr.size)
transformed = arr.reshape(-1)
transformed = np.expand_dims(transformed, axis=0)

result = savedModel.predict(transformed)

print(result[0])

64
3


Anbindung an Anvil:

In [4]:
import anvil.server
anvil.server.connect("AWL6UXNZCB5SDHRQRJVR75A2-637E3KHVNIOOIRXR")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment" as SERVER


In [5]:
import anvil.media
import PIL
from PIL import Image

@anvil.server.callable

def classifyImage(file):
    with anvil.media.TempFile(file) as filename:
        img = Image.open(filename)
    
    #Das Bild Skalieren und transformieren, so dass wir das
    #gleiche Datenformat haben wie beim Training!  
    img = PIL.ImageOps.grayscale(img)
    img.thumbnail((8,8), Image.Resampling.BICUBIC)
    arr = np.array(img)
    
    arr = arr/255 #auf 0-1 skalieren
    arr = 1 - arr #invertieren
    arr = arr*16 #auf 0-16 skalieren, da der dunkelste Wert in den Bilder im MINT Datensatz 16 ist
    transformed = arr.reshape(-1)
    transformed = np.expand_dims(transformed, axis=0)

    #den Klassifizierer mit den Bilddaten aufrufen
    result = savedModel.predict(transformed)

    return (float(result[0]))

